# Exemplos de Machine Learning - Treinamento AWS SageMaker

Este notebook contém exemplos práticos para o treinamento de Machine Learning com AWS SageMaker.

**Conteúdo:**
1. Análise Exploratória de Dados (EDA) Univariada
2. Modelo XGBoost para Regressão

---

## 1. Análise Exploratória de Dados (EDA) Univariada

A análise univariada examina cada variável individualmente para entender suas características:
- Estatísticas descritivas (média, mediana, desvio padrão)
- Assimetria (skewness)
- Curtose (kurtosis)

### 1.1 Importar Bibliotecas

In [ ]:
import pandas as pd
from scipy.stats import skew, kurtosis

### 1.2 Criar DataFrame de Exemplo

In [ ]:
# Criar um DataFrame de exemplo
dados = pd.DataFrame({
    'Idade': [25, 30, 22, 35, 28, 40, 27, 23, 32, 29],
    'Rendimento': [1800, 2400, 1500, 3100, 2000, 4000, 1900, 1700, 2800, 2200]
})

dados

### 1.3 Estatísticas Descritivas Básicas

In [ ]:
# Estatísticas descritivas básicas
print("Estatísticas descritivas básicas:")
dados.describe()

### 1.4 Coeficiente de Assimetria (Skewness)

O coeficiente de assimetria mede a simetria da distribuição:
- **= 0**: Distribuição simétrica
- **> 0**: Assimetria positiva (cauda à direita)
- **< 0**: Assimetria negativa (cauda à esquerda)

In [ ]:
# Calcular coeficiente de assimetria
print("Coeficiente de assimetria (skewness):")
print(f"Idade: {skew(dados['Idade']):.2f}")
print(f"Rendimento: {skew(dados['Rendimento']):.2f}")

### 1.5 Coeficiente de Curtose (Kurtosis)

A curtose mede o "achatamento" da distribuição:
- **= 3**: Distribuição normal (mesocúrtica)
- **> 3**: Caudas pesadas (leptocúrtica)
- **< 3**: Caudas leves (platicúrtica)

In [ ]:
# Coeficiente de achatamento (kurtosis)
print("Coeficiente de achatamento (kurtosis):")
print(f"Idade: {kurtosis(dados['Idade'], fisher=False):.2f}")
print(f"Rendimento: {kurtosis(dados['Rendimento'], fisher=False):.2f}")

print("\nExcesso de kurtosis (kurtosis-3):")
print(f"Idade: {kurtosis(dados['Idade'], fisher=True):.2f}")
print(f"Rendimento: {kurtosis(dados['Rendimento'], fisher=True):.2f}")

---

## 2. Modelo XGBoost para Regressão

Nesta seção, vamos treinar um modelo XGBoost para prever valores de imóveis usando o dataset California Housing.

**XGBoost (eXtreme Gradient Boosting)** é um algoritmo de ensemble baseado em árvores de decisão, muito utilizado em competições de ML devido à sua eficiência e performance.

### 2.1 Importar Bibliotecas

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)

### 2.2 Carregar o Dataset

O dataset **California Housing** contém informações sobre distritos da Califórnia:
- MedInc: Renda média
- HouseAge: Idade média das casas
- AveRooms: Média de quartos
- AveBedrms: Média de quartos de dormir
- Population: População
- AveOccup: Média de ocupantes
- Latitude/Longitude: Localização

**Target:** MedHouseVal (valor médio das casas em $100.000)

In [ ]:
# Carregar o conjunto de dados de habitação da Califórnia
california = fetch_california_housing()
X = pd.DataFrame(california.data, columns=california.feature_names)
y = pd.Series(california.target, name='MedHouseVal')

print(f"Shape do dataset: {X.shape}")
print(f"\nFeatures: {list(X.columns)}")
X.info()

In [ ]:
# Visualizar primeiras linhas
X.head()

### 2.3 Divisão Treino/Teste

Dividimos os dados em:
- **80%** para treino
- **20%** para teste

In [ ]:
# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Dados de treino: {X_train.shape[0]} registros")
print(f"Dados de teste: {X_test.shape[0]} registros")

### 2.4 Configurar e Treinar o Modelo XGBoost

**Hiperparâmetros principais:**
- `n_estimators`: Número de árvores
- `max_depth`: Profundidade máxima das árvores
- `learning_rate`: Taxa de aprendizado
- `subsample`: Fração de amostras por árvore
- `colsample_bytree`: Fração de features por árvore

In [ ]:
# Modelo XGBoost para regressão
xgb_reg = xgb.XGBRegressor(
    objective="reg:squarederror",
    n_estimators=200,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Treino
print("Treinando modelo XGBoost...")
xgb_reg.fit(X_train, y_train)
print("Treino concluído!")

### 2.5 Realizar Previsões

In [ ]:
# Previsões
y_pred = xgb_reg.predict(X_test)

# Comparar valores reais vs previstos
comparacao = pd.DataFrame({
    'Real': y_test.values[:10],
    'Previsto': y_pred[:10],
    'Diferença': abs(y_test.values[:10] - y_pred[:10])
})
comparacao

### 2.6 Avaliar o Desempenho do Modelo

**Métricas de avaliação:**
- **MSE** (Mean Squared Error): Erro quadrático médio
- **MAE** (Mean Absolute Error): Erro absoluto médio
- **MAPE** (Mean Absolute Percentage Error): Erro percentual absoluto médio
- **R²** (Coeficiente de Determinação): Quanto da variância é explicada pelo modelo

In [ ]:
# Avaliação do desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
r2 = r2_score(y_test, y_pred)

print("=" * 40)
print("Avaliação do Modelo XGBoost (Regressão)")
print("=" * 40)
print(f"MSE : {mse:.3f}")
print(f"MAE : {mae:.3f}")
print(f"MAPE: {mape:.2f}%")
print(f"R²  : {r2:.3f}")
print("=" * 40)

### 2.7 Importância das Variáveis

O XGBoost calcula automaticamente a importância de cada feature para as previsões.

In [ ]:
# Criar tabela de importância das variáveis
feature_importance = pd.DataFrame({
    "Feature": X.columns,
    "Importance": xgb_reg.feature_importances_
}).sort_values(by="Importance", ascending=False)

print("Importância das Variáveis:")
feature_importance

In [ ]:
# Gráfico das features mais importantes
plt.figure(figsize=(10, 6))
plt.barh(
    feature_importance["Feature"],
    feature_importance["Importance"],
    color='steelblue'
)
plt.gca().invert_yaxis()
plt.xlabel("Importância")
plt.ylabel("Variável")
plt.title("Importância das Variáveis - XGBoost")
plt.tight_layout()
plt.show()

---

## Conclusão

Neste notebook, exploramos:

1. **EDA Univariada**: Análise de estatísticas descritivas, assimetria e curtose
2. **XGBoost para Regressão**: Treinamento e avaliação de um modelo de ML

### Próximos Passos

- Aplicar esses conceitos ao dataset `no-shows.csv`
- Explorar treinamento gerenciado com SageMaker
- Utilizar algoritmos built-in do SageMaker